# How many Restless are there in the Necropolis of Warsaw? (part 1)

## Intro

Actually, there are two questions:

**AoD)** What is the **_age of death_** of how many wraiths across the population of Necropolis? AoD alters demeanor & nature of wraiths' psyche & shadow.

**DoD)** What is the **_date of death_** of how many wraiths? DoD defines culture context of a character and certain aspects of his/her social position.

Both questions share the same demographic model of getting to the Shadowlands, migrations & depopulation (Ascension/Oblivion/decorpsing) and differs only in entry data. In this part of the article, let's take **DoD** on the workshop and try to develop a prototypical model of it.

## DoD problem

In [ ]:
# lets do some math here
